In [0]:
# Example 1: learning about spark and anomaly detection of a temperature dataset.
# Dataset: datasets/temperature.csv
# Author: Humberto Bianchini

In [0]:
# 1) Importing all necessary libraries and Spark session creation.
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import monotonically_increasing_id



spark = SparkSession.builder.appName("AnomalyDetection").getOrCreate()

In [0]:
# 2) Reading and showing the first rows of the dataset.
datasetpath = "/FileStore/tables/temperature.csv"
data = spark.read.format("csv").options(header="true", inferschema="true").load(datasetpath)
data.show(5, False)

+-------------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----------------+-------------+-----+---------+---------+
datetime |Vancouver |Portland |San Francisco|Seattle |Los Angeles |San Diego |Las Vegas |Phoenix |Albuquerque |Denver |San Antonio |Dallas |Houston |Kansas City |Minneapolis |Saint Louis |Chicago |Nashville |Indianapolis |Atlanta |Detroit |Jacksonville |Charlotte |Miami |Pittsburgh |Toronto |Philadelphia |New York |Montreal |Boston |Beersheba |Tel Aviv District|Eilat |Haifa|Nahariyya|Jerusalem|
+-------------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----------------+-------------+-----+---------+---------+
2012-10-01 12:00:00|null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |null |309.1 |null |null |null |
2012-10-01 13:00:00|284.63 |282.08 |289.48 |281.8 |291.87 |291.53 |293.41 |296.6 |285.12 |284.61 |289.29 |289.74 |288.27 |289.98 |286.87 |286.18 |284.01 |287.41 |283.85 |294.03 |284.03 |298.17 |288.65 |299.72 |281.0 |286.26 |285.63 |288.22 |285.83 |287.17 |307.59 |305.47 |310.58 |304.4|304.4 |303.5 |
2012-10-01 14:00:00|284.62904131 |282.083251974|289.474992813|281.797216632|291.868185522|291.533500952|293.403141271|296.608508543|285.154558187|284.607305531|289.303648787|289.762974207|288.297575758|289.997635363|286.893635588|286.185246029|284.054690974|287.42136028 |283.889393939|294.03534141 |284.069789234|298.205229759|288.650172214|299.732517698|281.024767377|286.262540958|285.663207797|288.24767617 |285.834649953|287.186092094|307.59 |304.31 |310.495769231|304.4|304.4 |303.5 |
2012-10-01 15:00:00|284.626997923|282.091866475|289.460618112|281.789832606|291.862844459|291.543355079|293.392177052|296.631487354|285.233951595|284.5999178 |289.338496754|289.830766948|288.334343434|290.038150556|286.951400772|286.199194111|284.177412183|287.454636935|283.941919192|294.049702185|284.173964682|298.299595186|288.650581705|299.76657946 |281.088318736|286.269518418|285.756824139|288.326939663|285.847789539|287.23167159 |307.391512749|304.281841331 |310.411538462|304.4|304.4 |303.5 |
2012-10-01 16:00:00|284.624954535|282.100480976|289.446243412|281.78244858 |291.857503395|291.553209206|293.381212832|296.654466164|285.313345004|284.59253007 |289.373344722|289.89855969 |288.371111111|290.07866575 |287.009165955|286.213142193|284.300133393|287.48791359 |283.994444444|294.064062959|284.278140131|298.393960613|288.650991196|299.800641223|281.151870096|286.276495879|285.85044048 |288.406203155|285.860929124|287.277251086|307.145199718|304.238014609 |310.327307692|304.4|304.4 |303.5 |
+-------------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----------------+-------------+-----+---------+---------+
only showing top 5 row

In [0]:
# 3) Showing the dataset columns.
data.columns

Out[132]: ['datetime',
 'Vancouver',
 'Portland',
 'San Francisco',
 'Seattle',
 'Los Angeles',
 'San Diego',
 'Las Vegas',
 'Phoenix',
 'Albuquerque',
 'Denver',
 'San Antonio',
 'Dallas',
 'Houston',
 'Kansas City',
 'Minneapolis',
 'Saint Louis',
 'Chicago',
 'Nashville',
 'Indianapolis',
 'Atlanta',
 'Detroit',
 'Jacksonville',
 'Charlotte',
 'Miami',
 'Pittsburgh',
 'Toronto',
 'Philadelphia',
 'New York',
 'Montreal',
 'Boston',
 'Beersheba',
 'Tel Aviv District',
 'Eilat',
 'Haifa',
 'Nahariyya',
 'Jerusalem']

In [0]:
# 4) Showing the dataset rows count.
data.count()

Out[133]: 45253

In [0]:
# 5) Choosing one city to analysis (Vancouver in this example) and showing the details.
dataAnalysis = data.select('datetime','Vancouver')
dataAnalysis.show(5)

+-------------------+-------------+
 datetime| Vancouver|
+-------------------+-------------+
2012-10-01 12:00:00| null|
2012-10-01 13:00:00| 284.63|
2012-10-01 14:00:00| 284.62904131|
2012-10-01 15:00:00|284.626997923|
2012-10-01 16:00:00|284.624954535|
+-------------------+-------------+
only showing top 5 rows

In [0]:
# 6) Filtering only data that don't have null values.
dataNotNull = dataAnalysis.filter(col('Vancouver').isNotNull())
dataNotNull.show(5)

+-------------------+-------------+
 datetime| Vancouver|
+-------------------+-------------+
2012-10-01 13:00:00| 284.63|
2012-10-01 14:00:00| 284.62904131|
2012-10-01 15:00:00|284.626997923|
2012-10-01 16:00:00|284.624954535|
2012-10-01 17:00:00|284.622911147|
+-------------------+-------------+
only showing top 5 rows

In [0]:
# 7) Inserting index for the data.
df_plots = dataNotNull.withColumn("index", monotonically_increasing_id())
df_plots.show(5)

+-------------------+-------------+-----+
 datetime| Vancouver|index|
+-------------------+-------------+-----+
2012-10-01 13:00:00| 284.63| 0|
2012-10-01 14:00:00| 284.62904131| 1|
2012-10-01 15:00:00|284.626997923| 2|
2012-10-01 16:00:00|284.624954535| 3|
2012-10-01 17:00:00|284.622911147| 4|
+-------------------+-------------+-----+
only showing top 5 rows

In [0]:
# 8) Calculating the "Mean" and "Standard Deviation" of data.
from pyspark.sql.functions import avg, stddev
list_stats = dataNotNull.select(avg(col('Vancouver')).alias('mean'),stddev(col('Vancouver')).alias('stdDev')).collect()
mean = list_stats[0]['mean']
dev = list_stats[0]['stdDev']
print(f"Mean: {mean}")
print(f"Standard deviation: {dev}")

Mean: 283.86265374154124
Standard deviation: 6.640130616770929

In [0]:
# 9) Creating a new dataframe with the stats.

df_stats = dataNotNull.select(avg('Vancouver').alias('mean'), stddev('Vancouver').alias('stdDev'))
df_stats.show()

+------------------+-----------------+
 mean| stdDev|
+------------------+-----------------+
283.86265374154124|6.640130616770929|
+------------------+-----------------+

In [0]:
# 10) Describing the dataset
dataNotNull.describe().show()

+-------+-------------------+------------------+
summary| datetime| Vancouver|
+-------+-------------------+------------------+
 count| 44458| 44458|
 mean| null|283.86265374154124|
 stddev| null| 6.640130616770929|
 min|2012-10-01 13:00:00| 245.15|
 max|2017-10-28 00:00:00| 307.0|
+-------+-------------------+------------------+

In [0]:
# 11) Setting two functions to calculate anomalies.
from pyspark.sql.types import DoubleType, IntegerType
def distance(x):
  mean = 283.8626
  dev = 6.6401
  return ((x - mean)/dev)


def anomalies(x):
  dev = 6.6401
  if (x > 2):
    return 1
  else:
    return 0

  

distance_udf_double = udf(lambda z: distance(z), DoubleType())
anomalies_udf_int = udf(lambda z: anomalies(z), IntegerType())

In [0]:
# 12) Calculating the distances between a temperature value and the mean.
data_new = dataNotNull.select('Vancouver',distance_udf_double('Vancouver').alias('distance'))
data_new.show(5)

+-------------+-------------------+
 Vancouver| distance|
+-------------+-------------------+
 284.63|0.11557054863631709|
 284.62904131|0.11542616978660031|
284.626997923|0.11511843541512777|
284.624954535| 0.1148107008930567|
284.622911147|0.11450296637099419|
+-------------+-------------------+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import abs
data_new=data_new.select('Vancouver','distance', abs(col('distance')).alias("ABSdistance"))
data_new.show(5)

+-------------+-------------------+-------------------+
 Vancouver| distance| ABSdistance|
+-------------+-------------------+-------------------+
 284.63|0.11557054863631709|0.11557054863631709|
 284.62904131|0.11542616978660031|0.11542616978660031|
284.626997923|0.11511843541512777|0.11511843541512777|
284.624954535| 0.1148107008930567| 0.1148107008930567|
284.622911147|0.11450296637099419|0.11450296637099419|
+-------------+-------------------+-------------------+
only showing top 5 rows

In [0]:
data_new = data_new.select('Vancouver','distance', "ABSdistance", anomalies_udf_int("ABSdistance").alias("isAnomaly"))
data_new.show(10)

+-------------+-------------------+-------------------+---------+
 Vancouver| distance| ABSdistance|isAnomaly|
+-------------+-------------------+-------------------+---------+
 284.63|0.11557054863631709|0.11557054863631709| 0|
 284.62904131|0.11542616978660031|0.11542616978660031| 0|
284.626997923|0.11511843541512777|0.11511843541512777| 0|
284.624954535| 0.1148107008930567| 0.1148107008930567| 0|
284.622911147|0.11450296637099419|0.11450296637099419| 0|
284.620867759| 0.1141952318489231| 0.1141952318489231| 0|
284.618824371|0.11388749732685204|0.11388749732685204| 0|
284.616780983|0.11357976280478096|0.11357976280478096| 0|
284.614737595| 0.1132720282827099| 0.1132720282827099| 0|
284.612694207|0.11296429376063881|0.11296429376063881| 0|
+-------------+-------------------+-------------------+---------+
only showing top 10 rows

In [0]:
# 13) Showing the found anomalies.
data_new.filter(col("isAnomaly")>0).show(10)

+---------+-------------------+------------------+---------+
Vancouver| distance| ABSdistance|isAnomaly|
+---------+-------------------+------------------+---------+
 270.42|-2.0244574629900107|2.0244574629900107| 1|
 269.81|-2.1163235493441337|2.1163235493441337| 1|
 269.65| -2.140419571994399| 2.140419571994399| 1|
 269.67|-2.1374075691631105|2.1374075691631105| 1|
 269.67|-2.1374075691631105|2.1374075691631105| 1|
 270.52|-2.0093974488336026|2.0093974488336026| 1|
 269.15|-2.2157196427764654|2.2157196427764654| 1|
 267.15|-2.5169199259047317|2.5169199259047317| 1|
 266.15| -2.667520067468865| 2.667520067468865| 1|
 266.15| -2.667520067468865| 2.667520067468865| 1|
+---------+-------------------+------------------+---------+
only showing top 10 rows

In [0]:
display(dataNotNull.select("Vancouver"))

Vancouver
284.63
284.62904131
284.626997923
284.624954535
284.622911147
284.620867759
284.618824371
284.616780983
284.614737595
284.612694207


In [0]:
display(dataNotNull.select("Vancouver"))

Vancouver
284.63
284.62904131
284.626997923
284.624954535
284.622911147
284.620867759
284.618824371
284.616780983
284.614737595
284.612694207
